<a href="https://colab.research.google.com/github/Harshithaukp/camouflage-object-detection/blob/main/camouflage%20object%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 864.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

Using RGB model for thermal detection

0: 480x640 (no detections), 164.5ms
Speed: 2.8ms preprocess, 164.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 160.3ms
Speed: 2.7ms preprocess, 160.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
Detection Results:


In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import os
import urllib.request
import matplotlib.pyplot as plt

class FusionYOLODetector:
    def __init__(self, rgb_model_path='yolov8n.pt', thermal_model_path=None):
        # Download RGB model
        self.download_model('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt', 'yolov8n.pt')

        # Load models
        self.rgb_model = YOLO(rgb_model_path)
        self.thermal_model = self.load_thermal_model(thermal_model_path)

        # Detection parameters
        self.rgb_conf = 0.5
        self.thermal_conf = 0.3
        self.iou_threshold = 0.5
        self.classes = [0]  # Person class

        # Fusion parameters
        self.temperature_threshold = 100
        self.size_ratio_threshold = 0.7
        self.camouflage_threshold = 0.8

        # Visualization parameters
        self.camouflage_color = (0, 0, 255)  # Red
        self.normal_color = (0, 255, 0)       # Green
        self.suspected_color = (0, 165, 255)  # Orange
        self.rgb_only_color = (255, 0, 0)     # Blue
        self.thermal_only_color = (255, 255, 0) # Yellow
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.font_scale = 0.7
        self.thickness = 2

    def load_thermal_model(self, thermal_model_path):
        """Load thermal-specific model if available"""
        if thermal_model_path and os.path.exists(thermal_model_path):
            print("Loaded thermal-specific YOLO model")
            return YOLO(thermal_model_path)
        else:
            print("Using RGB model for thermal detection")
            return self.rgb_model

    def download_model(self, url, filename):
        """Download model if not available locally"""
        if not os.path.exists(filename):
            print(f"Downloading {filename}...")
            try:
                urllib.request.urlretrieve(url, filename)
                print("Download completed!")
            except Exception as e:
                print(f"Download failed: {e}")

    def detect_camouflage(self, rgb_image, thermal_image=None):
        """
        Main detection pipeline:
        1. Process RGB and thermal images separately
        2. Fuse detections
        3. Return annotated images for both modalities
        """
        # Generate simulated thermal if not provided
        if thermal_image is None:
            thermal_image = self.generate_simulated_thermal(rgb_image)

        # Preprocess thermal image
        thermal_vis = self.visualize_thermal(thermal_image)
        thermal_for_model = self.prepare_thermal_for_model(thermal_image)

        # Run detections
        rgb_boxes = self.detect_with_model(rgb_image, self.rgb_model, "RGB")
        thermal_boxes = self.detect_with_model(thermal_for_model, self.thermal_model, "Thermal")

        # Create base images for annotation
        rgb_annotated = rgb_image.copy()
        thermal_annotated = thermal_vis.copy()

        # Visualize individual detections
        self.visualize_detections(rgb_annotated, rgb_boxes, self.rgb_only_color, "RGB")
        self.visualize_detections(thermal_annotated, thermal_boxes, self.thermal_only_color, "Thermal")

        # Fuse detections and annotate
        results, fusion_img = self.fuse_detections(
            rgb_image.copy(),
            thermal_vis.copy(),
            rgb_boxes,
            thermal_boxes
        )

        # Create comparison image
        comparison_img = self.create_comparison_image(
            rgb_annotated,
            thermal_annotated,
            fusion_img
        )

        return results, comparison_img

    def detect_with_model(self, image, model, model_name):
        """Run detection with a specific model"""
        try:
            detections = model(image,
                               conf=self.rgb_conf if model_name=="RGB" else self.thermal_conf,
                               iou=self.iou_threshold,
                               classes=self.classes)[0]
            return self.process_detections(detections)
        except Exception as e:
            print(f"{model_name} detection failed: {e}")
            return []

    def prepare_thermal_for_model(self, thermal_image):
        """Convert thermal image to 3-channel for model input"""
        if len(thermal_image.shape) == 2:
            return cv2.cvtColor(thermal_image, cv2.COLOR_GRAY2BGR)
        return thermal_image

    def generate_simulated_thermal(self, rgb_image):
        """Create thermal simulation with human heat signatures"""
        gray = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY).astype(np.float32)
        h, w = gray.shape

        # Add human heat signatures
        for _ in range(np.random.randint(1, 3)):
            cx = np.random.randint(w//4, 3*w//4)
            cy = np.random.randint(h//4, 3*h//4)
            radius = np.random.randint(20, min(w, h)//6)

            # Create heat source
            y, x = np.ogrid[:h, :w]
            mask = (x - cx)**2 + (y - cy)**2 <= radius**2
            gray[mask] = np.clip(gray[mask] + 50 + np.random.randn()*10, 0, 255)

        return np.clip(gray, 0, 255).astype(np.uint8)

    def process_detections(self, detections):
        """Convert detections to standardized format"""
        boxes = []
        if detections is None:
            return boxes

        for box in detections.boxes:
            if box.xyxy.numel() > 0:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                conf = box.conf.item()
                cls_id = int(box.cls.item())
                boxes.append([x1, y1, x2, y2, conf, cls_id])
        return boxes

    def visualize_detections(self, image, boxes, color, label_prefix):
        """Draw detections on image with color coding"""
        for box in boxes:
            x1, y1, x2, y2, conf, cls_id = box
            cv2.rectangle(image, (x1, y1), (x2, y2), color, self.thickness)
            text = f"{label_prefix}: {conf:.2f}"
            cv2.putText(image, text, (x1, y1-10),
                       self.font, self.font_scale, color, self.thickness)

    def fuse_detections(self, rgb_img, thermal_img, rgb_boxes, thermal_boxes):
        """Fuse detections and annotate images"""
        results = []
        fusion_img = cv2.addWeighted(rgb_img, 0.7, thermal_img, 0.3, 0)

        # Process thermal detections
        for t_box in thermal_boxes:
            tx1, ty1, tx2, ty2, t_conf, t_cls = t_box

            # Find matching RGB detection
            best_match, max_iou = None, 0
            for r_box in rgb_boxes:
                rx1, ry1, rx2, ry2, r_conf, r_cls = r_box
                iou = self.calculate_iou((tx1, ty1, tx2, ty2), (rx1, ry1, rx2, ry2))
                if iou > max_iou:
                    max_iou = iou
                    best_match = r_box

            # Determine detection type
            if max_iou > self.iou_threshold and best_match:
                _, _, _, _, r_conf, _ = best_match
                confidence_ratio = t_conf / (r_conf + 1e-5)

                if confidence_ratio > self.camouflage_threshold:
                    result_type = 'camouflage'
                    color = self.camouflage_color
                    label = f"Camouflage: T:{t_conf:.2f}/R:{r_conf:.2f}"
                else:
                    result_type = 'normal'
                    color = self.normal_color
                    label = f"Normal: T:{t_conf:.2f}/R:{r_conf:.2f}"
            else:
                result_type = 'suspected_camouflage'
                color = self.suspected_color
                label = f"Suspected: {t_conf:.2f}"
                r_conf = 0 # No RGB match

            # Add to results
            results.append({
                'type': result_type,
                'bbox': (tx1, ty1, tx2, ty2),
                'thermal_confidence': t_conf,
                'rgb_confidence': r_conf
            })

            # Draw on fusion image
            cv2.rectangle(fusion_img, (tx1, ty1), (tx2, ty2), color, self.thickness)
            cv2.putText(fusion_img, label, (tx1, ty1-10),
                       self.font, self.font_scale, color, self.thickness)

        return results, fusion_img

    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union"""
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2

        # Intersection coordinates
        xi1 = max(x1_1, x1_2)
        yi1 = max(y1_1, y1_2)
        xi2 = min(x2_1, x2_2)
        yi2 = min(y2_1, y2_2)

        if xi2 <= xi1 or yi2 <= yi1:
            return 0.0

        intersection = (xi2 - xi1) * (yi2 - yi1)
        area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
        area2 = (x2_2 - x1_2) * (y2_2 - y1_2)

        return intersection / (area1 + area2 - intersection + 1e-5)

    def visualize_thermal(self, thermal_img):
        """Convert thermal to pseudo-color visualization"""
        if len(thermal_img.shape) == 3:
            thermal_img = cv2.cvtColor(thermal_img, cv2.COLOR_BGR2GRAY)
        norm_img = cv2.normalize(thermal_img, None, 0, 255, cv2.NORM_MINMAX)
        return cv2.applyColorMap(norm_img.astype(np.uint8), cv2.COLORMAP_JET)

    def create_comparison_image(self, rgb_det, thermal_det, fusion_det):
        """Create side-by-side comparison of all results"""
        # Resize all images to same dimensions
        h, w = 480, 640
        rgb_det = cv2.resize(rgb_det, (w, h))
        thermal_det = cv2.resize(thermal_det, (w, h))
        fusion_det = cv2.resize(fusion_det, (w, h))

        # Create top row (individual detections)
        top_row = np.hstack((rgb_det, thermal_det))

        # Create bottom row (fusion)
        bottom_row = np.hstack((fusion_det, np.zeros((h, w, 3), dtype=np.uint8)))

        # Add titles
        top_row = self.add_title(top_row, "RGB Detection | Thermal Detection")
        bottom_row = self.add_title(bottom_row, "Fusion Detection | Legend")

        # Create legend
        self.add_legend(bottom_row[:, w:])

        return np.vstack((top_row, bottom_row))

    def add_title(self, image, text):
        """Add title to image"""
        h, w = image.shape[:2]
        titled = cv2.putText(image.copy(), text, (10, 30),
                            self.font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        return cv2.rectangle(titled, (0,0), (w-1, h-1), (200,200,200), 2)

    def add_legend(self, legend_img):
        """Add color legend to image"""
        colors = {
            "RGB Only": self.rgb_only_color,
            "Thermal Only": self.thermal_only_color,
            "Normal": self.normal_color,
            "Suspected": self.suspected_color,
            "Camouflage": self.camouflage_color
        }

        y = 50
        for label, color in colors.items():
            cv2.rectangle(legend_img, (30, y-20), (80, y), color, -1)
            cv2.putText(legend_img, label, (100, y-5),
                       self.font, 0.8, (255,255,255), 2)
            y += 50

def create_sample_image():
    """Create forest scene with camouflaged person"""
    img = np.zeros((480, 640, 3), dtype=np.uint8)
    img[:, :, 1] = 100  # Green background

    # Add trees
    for _ in range(50):
        x, y = np.random.randint(0, 640), np.random.randint(0, 480)
        r = np.random.randint(5, 30)
        cv2.circle(img, (x, y), r, (0, np.random.randint(120, 180), 0), -1)

    # Add camouflaged person (green rectangle)
    cv2.rectangle(img, (300, 200), (350, 300), (50, 80, 50), -1)
    return img

# Example Usage
if __name__ == "__main__":
    # Create sample images
    rgb_img = create_sample_image()

    # Initialize detector
    detector = FusionYOLODetector()

    # Detect camouflage
    results, comparison_img = detector.detect_camouflage(rgb_img)

    # Print and show results
    print("Detection Results:")
    for i, res in enumerate(results):
        print(f"Detection {i+1}:")
        print(f"  Type: {res['type']}")
        print(f"  BBox: {res['bbox']}")
        print(f"  Thermal Conf: {res['thermal_confidence']:.2f}")
        if 'rgb_confidence' in res:
            print(f"  RGB Conf: {res['rgb_confidence']:.2f}")

    # Display results
    plt.figure(figsize=(16, 12))
    plt.imshow(cv2.cvtColor(comparison_img, cv2.COLOR_BGR2RGB))
    plt.title("Camouflage Detection Results", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('detection_results.jpg', dpi=300, bbox_inches='tight')
    plt.show()